In [1]:
import psycopg2
from tabulate import tabulate

# Connection Details
SUPABASE_USER = "postgres.ylhsfopeerhruicwrdsl"
SUPABASE_PASSWORD = "Alekhya@28"
SUPABASE_HOST = "aws-0-us-east-1.pooler.supabase.com"
SUPABASE_PORT = "5432"
SUPABASE_DBNAME = "postgres"

def execute_query(query, connection):
    """Executes a SQL query and returns both the results and column names."""
    with connection.cursor() as cur:
        cur.execute(query)
        try:
            results = cur.fetchall()
            columns = [desc[0] for desc in cur.description]
        except psycopg2.ProgrammingError:
            results = None
            columns = []
        connection.commit()
    return results, columns

def main():
    # Connect to the Supabase PostgreSQL database
    conn = psycopg2.connect(
        host=SUPABASE_HOST,
        dbname=SUPABASE_DBNAME,
        user=SUPABASE_USER,
        password=SUPABASE_PASSWORD,
        port=SUPABASE_PORT
    )
    
    ### 1. Channel-Level Performance Query ###
    channel_query = """
    WITH spend_summary AS (
      SELECT 
        channel,
        SUM(spend) AS total_spend,
        SUM(impressions) AS total_impressions,
        SUM(clicks) AS total_clicks,
        SUM(installs) AS total_installs
      FROM marketing_spend
      GROUP BY channel
    ),
    conversion_summary AS (
      SELECT 
        channel,
        COUNT(*) FILTER (WHERE conversion = TRUE) AS total_conversions
      FROM user_touchpoints
      GROUP BY channel
    ),
    revenue_summary AS (
      SELECT 
        u.channel,
        SUM(r.revenue) AS total_revenue
      FROM users u
      JOIN revenue_and_rewards r ON u.user_id = r.user_id
      GROUP BY u.channel
    )
    SELECT 
      s.channel,
      s.total_spend,
      s.total_impressions,
      s.total_clicks,
      s.total_installs,
      COALESCE(c.total_conversions, 0) AS total_conversions,
      COALESCE(r.total_revenue, 0) AS total_revenue
    FROM spend_summary s
    LEFT JOIN conversion_summary c ON s.channel = c.channel
    LEFT JOIN revenue_summary r ON s.channel = r.channel;
    """
    
    channel_results, channel_columns = execute_query(channel_query, conn)
    print("Channel-Level Performance:")
    if channel_results:
        # Use floatfmt to format all floats with two decimal places
        print(tabulate(channel_results, headers=channel_columns, tablefmt="psql", floatfmt=".2f"))
    else:
        print("No channel-level results returned.")
    
    ### 2. Campaign-Level Performance & ROAS Query ###
    campaign_query = """
    WITH spend_summary AS (
      SELECT 
        campaign,
        SUM(spend) AS total_spend,
        SUM(impressions) AS total_impressions,
        SUM(clicks) AS total_clicks,
        SUM(installs) AS total_installs
      FROM marketing_spend
      GROUP BY campaign
    ),
    conversion_summary AS (
      SELECT 
        campaign,
        COUNT(*) FILTER (WHERE conversion = TRUE) AS total_conversions
      FROM user_touchpoints
      GROUP BY campaign
    ),
    revenue_summary AS (
      SELECT 
        u.campaign,
        SUM(r.revenue) AS total_revenue
      FROM users u
      JOIN revenue_and_rewards r ON u.user_id = r.user_id
      GROUP BY u.campaign
    )
    SELECT 
      s.campaign,
      s.total_spend,
      s.total_impressions,
      s.total_clicks,
      s.total_installs,
      COALESCE(c.total_conversions, 0) AS total_conversions,
      COALESCE(r.total_revenue, 0) AS total_revenue,
      CASE 
        WHEN s.total_spend > 0 THEN ROUND((COALESCE(r.total_revenue, 0) / s.total_spend)::numeric, 2)
        ELSE NULL
      END AS roas
    FROM spend_summary s
    LEFT JOIN conversion_summary c ON s.campaign = c.campaign
    LEFT JOIN revenue_summary r ON s.campaign = r.campaign;
    """
    
    campaign_results, campaign_columns = execute_query(campaign_query, conn)
    print("\nCampaign-Level Performance & ROAS:")
    if campaign_results:
        print(tabulate(campaign_results, headers=campaign_columns, tablefmt="psql", floatfmt=".2f"))
    else:
        print("No campaign-level results returned.")
    
    ### 3. Denormalized Materialized View for Users ###
    # Create the materialized view if it doesn't exist
    create_mv_query = """
    CREATE MATERIALIZED VIEW IF NOT EXISTS user_metrics AS
    SELECT 
      u.user_id,
      u.installed_at,
      u.channel,
      u.campaign,
      COALESCE(SUM(r.revenue), 0) AS total_revenue,
      COALESCE(SUM(r.reward_to_user), 0) AS total_rewards,
      COUNT(ut.id) AS touchpoints_count
    FROM users u
    LEFT JOIN revenue_and_rewards r ON u.user_id = r.user_id
    LEFT JOIN user_touchpoints ut ON u.user_id = ut.user_id
    GROUP BY u.user_id, u.installed_at, u.channel, u.campaign;
    """
    
    print("\nCreating (or refreshing) the 'user_metrics' materialized view...")
    execute_query(create_mv_query, conn)
    print("Materialized view 'user_metrics' created (or already exists).")
    
    # Query the materialized view
    query_mv = "SELECT * FROM user_metrics LIMIT 10;"
    mv_results, mv_columns = execute_query(query_mv, conn)
    print("\nUser Metrics (from materialized view):")
    if mv_results:
        print(tabulate(mv_results, headers=mv_columns, tablefmt="psql", floatfmt=".2f"))
    else:
        print("No results returned from the materialized view.")
    
    # Close the connection when done
    conn.close()

if __name__ == "__main__":
    main()


Channel-Level Performance:
+------------+---------------+---------------------+----------------+------------------+---------------------+-----------------+
| channel    |   total_spend |   total_impressions |   total_clicks |   total_installs |   total_conversions |   total_revenue |
|------------+---------------+---------------------+----------------+------------------+---------------------+-----------------|
| affiliate  |       9317.01 |           517110.00 |       19806.00 |          3885.00 |                5234 |        72180.10 |
| facebook   |      29205.11 |          2280049.00 |       62934.00 |          8883.00 |                5283 |       157744.00 |
| google_ads |      40751.35 |          1728387.00 |       59581.00 |         10674.00 |                5243 |       155118.00 |
| instagram  |      23294.96 |          2342705.00 |       57677.00 |          6565.00 |                5168 |       108445.00 |
| twitter    |      13950.93 |          2731093.00 |       53713.00 | 